## 데이타 증강 - 일반대화는 증강하지 않는다.

In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [2]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
train_data_path = os.path.join(data_dir, 'train_no_dup.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,3748,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요.\n그건 싫어요.\n사오라면 사와야지 인턴주제.\n그러니까요...
1,3149,직장 내 괴롭힘 대화,자 점심식사들 합시다\n 와 점심시간이다. 과장님 밥 어디서 먹죠 오늘은?\n 대리...
2,1866,협박 대화,어이 이쁜이 시간이 지났는데 약속을 안지키네\n네.엄마가 아프셔서.\n그래 엄마가 ...
3,1493,기타 괴롭힘 대화,우리 귀여운 철수야 여기 돈 줄테니까 핫도그랑 콜라좀 사다줄래? 남은 돈은 너 가져...
4,366,직장 내 괴롭힘 대화,어이 김대리\n예 이부장님\n오늘 하라고 했던 서류 정리 다 끝냈어?\n아.그게 아...


In [3]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 3846
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3846 entries, 0 to 3845
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3846 non-null   int64 
 1   class         3846 non-null   object
 2   conversation  3846 non-null   object
dtypes: int64(1), object(2)
memory usage: 90.3+ KB


**한국어 불용어 사전**

In [4]:
# 불용어 제거
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    return sentence

In [5]:
import re

def preprocess_korean(sentence):
    # 한글 외 모든 문자 제거
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', sentence)
    # 연속된 공백 제거
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence


**동의어**

In [6]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

# 전체': ['전체', '전적', '전부', '총체', '전반']
# '인': ['인', '인간', '인물', '사람', '인류']
#print(wordnet)

In [7]:
# wordnet에서 동의어 리스트를 가져온다. 
# 동의어가 없을 수도 있다
def synonyms_from_wordnet(word):
    synonyms = []
    try:
        for list in wordnet[word] :
             synonyms.append(list)
    except KeyError:
        pass
    
    return synonyms

In [8]:
#'전체': ['전체', '전적', '전부', '총체', '전반']
synonyms = synonyms_from_wordnet( "전체" )
for word in synonyms : 
    print(word)

전체
전적
전부
총체
전반


In [9]:
# word list로 구성된 sentense를 동의어로 바꿔준다
def replace_to_synonyms(input_words, count): 
     
    # new_words
    new_words = []
    
    #
    word_list = list(set([word for word in input_words]))
    random.shuffle(word_list)
    
    replace_count = 0
    for word in word_list : 
        # 동의어 리스트를 가져온다. 
        synonyms = synonyms_from_wordnet( word )
        
        if len(synonyms) == 0 : continue
        
        synonym = random.choice(list(synonyms))
        if( word == synonym) : continue
            
        new_words = [synonym if wd == word else wd for wd in input_words]
        
        replace_count +=1
        
        if replace_count >= count : break
    
    # 
    #if len(new_words) > 0 : 
    #    sentence = ' '.join(new_words)
    #   new_words = sentence.split(" ")
    
       
    return new_words
               

In [10]:
def gen_sentence_by_swap_word(inputs_words, count):
    
    # new_words
    new_words = []
    
    #
#    word_list = list(set([word for word in input_words]))
#    random.shuffle(word_list)
    
    return new_words

In [11]:
def DataAugmentation(sentense, alpha_sr=0.2, aug_count=1):
    
    augmented_sentences = []
    
    # 한글만 처리
    sentense = preprocess_korean(sentense)
    sentense = sentense.split(' ')
    words = [word for word in sentense if word != ""]
    num_words = len(words)
       
    # 동의어로 단어 바꿔 문장 생성하기
    n_sr = max(1, int(alpha_sr*num_words))
    
    for _ in range(aug_count):
        a_words = replace_to_synonyms( words, n_sr )
          
        if( len(a_words) > 0 ):
            augmented_sentences.append(" ".join(a_words))
     
    
    
    # 생성된 문장들 shuffle
    random.shuffle(augmented_sentences)
    return augmented_sentences

In [12]:
new_class_list = []
new_conversation_list = []
new_idx_list = []

idx = len(train_data['conversation'])+1;
          
for i in range(len(train_data['conversation'])):
#for i in range(2):
    class_ = train_data['class'][i]
    
    num_aug = 3
    
    augmented_sentences = DataAugmentation(train_data['conversation'][i], aug_count=num_aug)
    
    if not augmented_sentences : # 리스트가 비어 있으면 True
        continue
        
    for j in range(min(num_aug, len(augmented_sentences))):
        #if not augmented_sentences[j] : continue;
        new_class_list.append( class_ )
        new_conversation_list.append(augmented_sentences[j])
        idx =idx+1
        new_idx_list.append(int(idx))

print(len(new_class_list))    
print(len(new_conversation_list)) 
        

8005
8005


In [13]:
train_data['conversation'][0:1]

0    지은씨 커피 좀 사와요.\n그건 싫어요.\n사오라면 사와야지 인턴주제.\n그러니까요...
Name: conversation, dtype: object

In [14]:
new_conversation_list[0:3]

['지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이거 권력 남용인 거 아세요 뭐 권력 남용 지은씨 회사 짤리고 싶어 제가 더러운 회사 나가고 만다 그 말에 원죄 질 수 있나 네 수고하세요',
 '지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이거 권력 남용인 거 아세요 뭐 권력 남용 지은씨 회사 짤리고 싶어 제가 더러운 회사 나가고 만다 그 말에 악행 질 수 있나 네 수고하세요',
 '지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이거 권력 남용인 거 아세요 뭐 권력 남용 지은씨 회사 짤리고 싶어 제가 더러운 회사 나가고 만다 그 말에 책임 명사 수 있나 네 수고하세요']

In [15]:
aug_data = pd.DataFrame([new_idx_list, new_class_list, new_conversation_list],
                  index=['idx','class','conversation'],
                 columns=[x for x in range(len(new_class_list))])
aug_data  = aug_data .transpose()

In [24]:
aug_data.head()

,idx,class,conversation
0,3848,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이...
1,3849,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이...
2,3850,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요 그건 싫어요 사오라면 사와야지 인턴주제 그러니까요 부장님 이...
3,3851,기타 괴롭힘 대화,우리 귀여운 철수야 여기 돈 줄테니까 핫도그랑 콜라좀 사다줄래 남은 돈은 너 가져 ...
4,3852,기타 괴롭힘 대화,우리 귀여운 철수야 여기 시가 줄테니까 핫도그랑 콜라좀 사다줄래 남은 돈은 너 가져...


In [25]:
aug_train_data = aug_data.sample(frac=1).reset_index(drop=True)

aug_train_data['idx'] = [x for x in range(len(aug_train_data))]
aug_train_data = aug_train_data.reset_index(drop=True)

In [27]:
aug_train_data.head()

,idx,class,conversation
0,0,협박 대화,그게 사실이야 죄송합니다 그게 솔직히 너가 그랬을 거라고 생각하지 않아 아닙니다 한...
1,1,직장 내 괴롭힘 대화,박사원 몇살이에요 예 몇살이길래 이거 하나 제대로 못해와 혹시 학벌 속였어요 아닙니...
2,2,기타 괴롭힘 대화,아 너 진짜 싫다 왜 몰라서 물어 너희 부모님이 부탁해서 너 챙겨주긴하는데 솔직히 ...
3,3,기타 괴롭힘 대화,님 그 호텔 좋아요 신고했어요 신고 안 될텐데 나 아이피추적도 안 돼요 왜 이러세요...
4,4,직장 내 괴롭힘 대화,저기 과장님 회식 자리에서 그때 죄송합니다 제가 술에 너무 취해서 시끄러워 어디서 ...


In [28]:
aug_train_data.tail()

,idx,class,conversation
8000,8000,기타 괴롭힘 대화,야 이것좀 빌리자 안돼 전에 가져갔던 무선이어폰도 안주고 있잖아 그건 내가 쓰고 있...
8001,8001,직장 내 괴롭힘 대화,영희씨 이거 이렇게 하면 어떻게 유해 나더러 어떻게 하라고 네 그거 어제 팀장님이 ...
8002,8002,기타 괴롭힘 대화,아 정글 씨발새끼 정글에 엄마 묻어놨냐 계속 찾아다니네 그니까 쉬발 지가 갱당해 뒤...
8003,8003,갈취 대화,너 이번주 용돈 받은거 다 내놔 나 학교 끝나고 여친이랑 오토바이 타고 부산 가야해...
8004,8004,기타 괴롭힘 대화,야 이새끼 봐바 오타쿠인가봐 뭔데뭔데 아 뭐냐 그림 이게 애인 미미쨩인가 그거냐 하...


In [29]:
print(len(aug_train_data))

8005


In [30]:
class_counts = aug_train_data['class'].value_counts()

print(class_counts)

갈취 대화          2287
기타 괴롭힘 대화      1991
협박 대화          1924
직장 내 괴롭힘 대화    1803
Name: class, dtype: int64


In [32]:
aug_train_data.to_csv('../../data/aug_csv/aug_synonym_train_only.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함

In [33]:
aug_data_path ="../../data/aug_csv/aug_synonym_train_only.csv"
aug_data = pd.read_csv(train_data_path)
aug_data.head()

,idx,class,conversation
0,3748,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요.\n그건 싫어요.\n사오라면 사와야지 인턴주제.\n그러니까요...
1,3149,직장 내 괴롭힘 대화,자 점심식사들 합시다\n 와 점심시간이다. 과장님 밥 어디서 먹죠 오늘은?\n 대리...
2,1866,협박 대화,어이 이쁜이 시간이 지났는데 약속을 안지키네\n네.엄마가 아프셔서.\n그래 엄마가 ...
3,1493,기타 괴롭힘 대화,우리 귀여운 철수야 여기 돈 줄테니까 핫도그랑 콜라좀 사다줄래? 남은 돈은 너 가져...
4,366,직장 내 괴롭힘 대화,어이 김대리\n예 이부장님\n오늘 하라고 했던 서류 정리 다 끝냈어?\n아.그게 아...
